# Evaluate trained slime agents


In [4]:
import sys
import os
PACKAGE_PARENT = '../'
sys.path.append(PACKAGE_PARENT)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import gym
import slimevolleygym
import pickle




In [15]:
eval_path = '../evaluate_data/match_data.pkl'
with open(eval_path, 'rb') as f:
    match_data = pickle.load(f)

# print(match_data.keys())
# new dict that stores, "match":{avg_rew, std}
match_stats = {}

for match_name in match_data.keys():
    rewards, _ = match_data[match_name]
    rewards = np.array(rewards)
    mean = np.mean(rewards)
    std = np.std(rewards)
    match_stats[match_name] = [mean, std]





In [16]:
for match_name in match_stats.keys():
    print('-'*10)
    print(match_name)
    mean, std = match_stats[match_name]
    print('{} +- {}'.format(mean, std))
    print('-'*10)

----------
PPO_Agent_vs_PPO_Agent
-0.034 +- 2.226846200347029
----------
----------
PPO_Agent_vs_Self-play_PPO_Agent
2.385 +- 2.159345965796125
----------
----------
PPO_Agent_vs_Bootstrapped_Self-play_PPO_Agent
0.417 +- 2.585558160243161
----------
----------
PPO_Agent_vs_BC_Agent
0.339 +- 1.7378374492454696
----------
----------
Self-play_PPO_Agent_vs_PPO_Agent
-2.378 +- 2.202978892318308
----------
----------
Self-play_PPO_Agent_vs_Self-play_PPO_Agent
-0.05 +- 2.7805574980568193
----------
----------
Self-play_PPO_Agent_vs_Bootstrapped_Self-play_PPO_Agent
-2.2 +- 2.1573131437044553
----------
----------
Self-play_PPO_Agent_vs_BC_Agent
-4.431 +- 0.7753960278464161
----------
----------
Bootstrapped_Self-play_PPO_Agent_vs_PPO_Agent
-0.492 +- 2.607285178111516
----------
----------
Bootstrapped_Self-play_PPO_Agent_vs_Self-play_PPO_Agent
2.081 +- 2.2082660618684513
----------
----------
Bootstrapped_Self-play_PPO_Agent_vs_Bootstrapped_Self-play_PPO_Agent
-0.11 +- 2.723215011709505
-----

In [19]:
# view baseline data

eval_path = '../evaluate_data/baseline_match_data.pkl'
with open(eval_path, 'rb') as f:
    match_data_baseline = pickle.load(f)

# print(match_data.keys())
# new dict that stores, "match":{avg_rew, std}
baseline_match_stats = {}

for baseline_match_name in match_data_baseline.keys():
    rewards, _ = match_data_baseline[baseline_match_name]
    rewards = np.array(rewards)
    mean = np.mean(rewards)
    std = np.std(rewards)
    baseline_match_stats[baseline_match_name] = [mean, std]

In [21]:
for match_name in baseline_match_stats.keys():
    print('-'*10)
    print(match_name)
    mean, std = baseline_match_stats[match_name]
    print('{} +- {}'.format(mean, std))
    print('-'*10)

----------
PPO_Agent_vs_Baseline
-0.627 +- 1.3228268972167145
----------
----------
Self-play_PPO_Agent_vs_Baseline
-4.704 +- 0.5480729878401235
----------
----------
Bootstrapped_Self-play_PPO_Agent_vs_Baseline
-2.975 +- 1.6444984037693682
----------
----------
BC_Agent_vs_Baseline
-0.751 +- 1.4829022219957726
----------
----------
Baseline_vs_PPO_Agent
0.625 +- 1.3499537029098443
----------
----------
Baseline_vs_Self-play_PPO_Agent
4.722 +- 0.5447164399942415
----------
----------
Baseline_vs_Bootstrapped_Self-play_PPO_Agent
2.999 +- 1.641035953292919
----------
----------
Baseline_vs_BC_Agent
0.759 +- 1.487588316705936
----------


In [28]:
from alphaslime.evaluate.eval_agents import EvaluateGameMA
import trained_agents.agents as TRAINED_AGENTS

# lets see the performance of baseline vs baseline
agent_right = TRAINED_AGENTS.load_baseline()
agent_left = TRAINED_AGENTS.load_baseline()
agent_right_name = 'Baseline'
agent_left_name = 'Baseline'

env_id = "SlimeVolley-v0"
env = gym.make(env_id)
base_dir_path = './'
RENDER = False
eps = 1000
running_avg_len = 100

gym_evaluator = EvaluateGameMA(agent_right, agent_left, env, base_dir_path, render=RENDER, time_delay=0)

# evaulate agent
rewards, avg_rewards_array = gym_evaluator.evaluate(eps, is_progress_bar=True, running_avg_len=running_avg_len)

data_name = agent_right_name + '_vs_' + agent_left_name
match_data[data_name] = [rewards, avg_rewards_array]
print(data_name)


100%|██████████| 1000/1000 [02:59<00:00,  5.57it/s]

Baseline_vs_Baseline


In [34]:
match_data[data_name] = [rewards, avg_rewards_array]

# print(match_data[data_name])

In [35]:
mean = np.mean(rewards)
std = np.std(rewards)
baseline_match_stats[data_name] = [mean, std]
print(data_name)
mean, std = baseline_match_stats[data_name]
print('{} +- {}'.format(mean, std))

Baseline_vs_Baseline
0.019 +- 1.2315189807713076
